<a href="https://colab.research.google.com/github/AdithyaJesuman/Image_classification/blob/main/Bankchurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#!pip install tensorflow
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
from tensorflow.keras.metrics import Precision, Recall

In [35]:
#link=https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling
df=pd.read_csv("Churn_Modelling.csv")
col=["RowNumber","CustomerId","Surname"]
df.drop(col,axis='columns',inplace=True)
df.columns[:]
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2055,469,Germany,Male,38,9,113599.42,1,0,0,11950.29,0
5681,529,Spain,Female,39,2,82766.43,1,1,1,122925.44,0
2653,730,Spain,Male,26,3,0.00,1,1,0,34542.41,0
3303,632,France,Female,44,3,133793.89,1,1,1,34607.14,1
7440,622,Spain,Female,47,6,142319.03,1,0,0,100183.05,0


In [36]:
def uq():
  for col in df.columns:
      print(f"{col}: {df[col].unique()}\n")
uq()

CreditScore: [619 608 502 699 850 645 822 376 501 684 528 497 476 549 635 616 653 587
 726 732 636 510 669 846 577 756 571 574 411 591 533 553 520 722 475 490
 804 582 472 465 556 834 660 776 829 637 550 698 585 788 655 601 656 725
 511 614 742 687 555 603 751 581 735 661 675 738 813 657 604 519 664 678
 757 416 665 777 543 506 493 652 750 729 646 647 808 524 769 730 515 773
 814 710 413 623 670 622 785 605 479 685 538 562 721 628 668 828 674 625
 432 770 758 795 686 789 589 461 584 579 663 682 793 691 485 650 754 535
 716 539 706 586 631 717 800 683 704 615 667 484 480 578 512 606 597 778
 514 525 715 580 807 521 759 516 711 618 643 671 689 620 676 572 695 592
 567 694 547 594 673 610 767 763 712 703 662 659 523 772 545 634 739 771
 681 544 696 766 727 693 557 531 498 651 791 733 811 707 714 782 775 799
 602 744 588 747 583 627 731 629 438 642 806 474 559 429 680 749 734 644
 626 649 805 718 840 630 654 762 568 613 522 737 648 443 640 540 460 593
 801 611 802 745 483 690 492 709 705 5

In [37]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)
df=pd.get_dummies(data=df,columns=['Geography'])
df.columns[:]
df.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
4646,805,1,26,8,42712.87,2,1,1,28861.69,0,False,True,False
225,670,0,37,4,170557.91,2,1,0,198252.88,0,True,False,False
1861,565,1,38,2,158651.29,2,1,1,179445.28,0,False,True,False
8002,590,0,37,1,0.00,2,0,0,133535.99,0,False,False,True
874,645,1,38,5,101430.30,2,0,1,4400.32,0,True,False,False


In [38]:
df['Balance']=pd.to_numeric(df['Balance'])
df['EstimatedSalary']=pd.to_numeric(df['EstimatedSalary'])
df.dtypes

,0
CreditScore,int64
Gender,int64
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64
HasCrCard,int64
IsActiveMember,int64
EstimatedSalary,float64
Exited,int64


In [39]:
scale=MinMaxScaler()
col=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
df[col]=scale.fit_transform(df[col])
df.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
4400,0.832,1,0.216216,0.6,0.424216,0.000000,0,1,0.689986,1,False,True,False
5102,0.624,0,0.027027,0.9,0.416539,0.333333,0,0,0.365515,0,False,False,True
8218,0.332,1,0.297297,0.9,0.000000,0.333333,0,1,0.166289,0,True,False,False
4897,0.698,0,0.297297,0.2,0.000000,0.000000,1,0,0.391916,0,False,False,True
7200,0.348,0,0.135135,0.1,0.372969,0.000000,1,1,0.258321,0,True,False,False


In [40]:
df.shape[1]

13

In [41]:
X_train,X_test,Y_train,Y_test=train_test_split(df.drop('Exited',axis='columns'),df['Exited'],test_size=0.2,random_state=5)
model=keras.Sequential([
    keras.layers.Dense(12,input_shape=(12,),activation='relu'),
    keras.layers.Dense(6,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',Precision(),Recall()])
model.fit(X_train,Y_train,epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6303 - loss: 0.6312 - precision_1: 0.1986 - recall_1: 0.2535
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7912 - loss: 0.4930 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8079 - loss: 0.4528 - precision_1: 0.5795 - recall_1: 0.0229
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8072 - loss: 0.4459 - precision_1: 0.5751 - recall_1: 0.1110
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8139 - loss: 0.4343 - precision_1: 0.6573 - recall_1: 0.1643
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8115 - loss: 0.4410 - precision_1: 0.6783 - recall_1: 0.1721
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8024 - loss: 0.4438 - precision_1: 0.6551 - recall_1: 0.2018
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8177 - loss: 0.4237 - precisi

In [42]:
model.evaluate(X_test,Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8527 - loss: 0.3517 - precision_1: 0.7175 - recall_1: 0.4287


[0.35132357478141785,
 0.8539999723434448,
 0.7424892783164978,
 0.42716050148010254]

In [44]:
yp = model.predict(X_test)
prediction=[]
for i in yp :
  if i>0.5 :
    prediction.append(1)
  else :
    prediction.append(0)
count = 0
Y_test=Y_test.to_numpy()
for i in range(100):
    if Y_test[i]== prediction[i]:
        count += 1
print(count)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
89


In [48]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(Y_test[:1000],prediction[:1000]))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91       794
           1       0.70      0.43      0.53       206

    accuracy                           0.84      1000
   macro avg       0.78      0.69      0.72      1000
weighted avg       0.83      0.84      0.83      1000

